In [1]:
import requests
import pandas as pd
import plotly.graph_objects as go

In [2]:
# Need to specify email to access API
header={'User-Agent': "XXXXX"}

In [3]:
# Get list of company cik numbers
tickers_cik=requests.get('https://www.sec.gov/files/company_tickers.json', headers=header)
tickers_cik=pd.json_normalize(pd.json_normalize(tickers_cik.json(), max_level=0).values[0])
cik_list = tickers_cik['cik_str'].astype(str).tolist()
ticker_list = tickers_cik['ticker'].astype(str).tolist()

In [4]:
# Access API
from sec_edgar_api import EdgarClient
edgar = EdgarClient(user_agent="<Sample Company Name> <Admin Contact>@<Sample Company Domain>")

In [6]:
# Create dictionary of financial metrics
financials_dict = {'GrossProfit': None, 'CostOfGoodsAndServicesSold': None, 'OperatingIncomeLoss': None, 
                   'IncomeTaxesPaidNet': None, 'NetIncomeLoss': None, 'OperatingExpenses': None, 
                   'ResearchAndDevelopmentExpense': None, 'SellingGeneralAndAdministrativeExpense': None}

# Function to format number into billions
def format_billion(value):
    return '$'+str(f'{value / 1e9:.1f}')+'B'

# Function to return Sankey chart
def apple_income_statement_metrics_2022():
    for j in range(len(financials_dict)): # Iterate through financials_dict
        edgar_data = edgar.get_company_facts(
            cik='320193')['facts']['us-gaap'][list(financials_dict.keys())[j]]['units']['USD'] # Use keys in financials_dict
        for k in range(len(edgar_data)): # Iterate through company data
            if edgar_data[k]['form']=='10-K' and edgar_data[k]['fy']==2022: # Specify form and fy
                if str(edgar_data[k]['start']).startswith('2021'): # Get correct fy value
                    financials_dict[list(financials_dict.keys())[j]]=edgar_data[k]['val']
                else:
                    continue
            else:
                continue                
    
    # Specify source and target links for Sankey chart
    link_source = [0, 0, 1, 1, 3, 3, 6, 6]
    link_target = [1, 2, 3, 6, 5, 4, 7, 8]
    
    # Specify the values for each link
    link_value = [financials_dict['GrossProfit'], financials_dict['CostOfGoodsAndServicesSold'], 
                  financials_dict['OperatingIncomeLoss'], financials_dict['OperatingExpenses'], 
                  financials_dict['NetIncomeLoss'], financials_dict['IncomeTaxesPaidNet'], 
                  financials_dict['ResearchAndDevelopmentExpense'], 
                  financials_dict['SellingGeneralAndAdministrativeExpense']]
    
    # Specify the value for each node
    node_labels = [f'Revenue: {format_billion((link_value[0]+link_value[1]))}', 
                   f'Gross Profit: {format_billion(link_value[0])}', 
                   f'Cost of Goods and Services Sold: {format_billion(link_value[1])}', 
                   f'Operating Profit: {format_billion(link_value[2])}', 
                   f'Taxes: {format_billion(link_value[5])}', 
                   f'Net Profit: {format_billion(link_value[4])}', 
                   f'Operating Expenses: {format_billion(link_value[3])}', 
                   f'R&D Expense: {format_billion(link_value[6])}', 
                   f'SG&A Expense: {format_billion(link_value[7])}']
    
    # Specify colors for nodes and links
    node_colors = ["rgba(180, 177, 177, 1)", "green", "rgba(250, 34, 13, 0.8)", "green", "rgba(250, 34, 13, 0.8)", "green", 
                   "rgba(250, 34, 13, 0.8)", "rgba(250, 34, 13, 0.8)", "rgba(250, 34, 13, 0.8)", "rgba(250, 34, 13, 0.8)"]
    link_colors = ["rgba(0, 128, 0, 0.3)", "rgba(255, 0, 0, 0.3)", "rgba(0, 128, 0, 0.3)", "rgba(255, 0, 0, 0.3)", 
                   "rgba(0, 128, 0, 0.3)", "rgba(255, 0, 0, 0.3)", "rgba(255, 0, 0, 0.3)", "rgba(255, 0, 0, 0.3)", 
                   "rgba(255, 0, 0, 0.3)"]
    
    # Create a sankey object with formatting
    sankey = go.Figure(data=[go.Sankey(
        valueformat = ".4s",
        valuesuffix = " (B)",
        node=dict(
            pad=15,
            thickness=20,
            line=dict(color='gray', width=0.5),
            label=node_labels,
            color=node_colors
        ),
        link=dict(
            source=link_source,
            target=link_target,
            value=link_value,
            label=link_value,
            color=link_colors
        ),
        textfont=dict(
            color="black"
        )
    )])
    
    sankey.update_layout(
        title_text="Apple FY22 Income Statement <br>Source: <a href='https://www.sec.gov/edgar/sec-api-documentation'>SEC EDGAR</a>",
        title_font=dict(size=20, color="dark green"),
        title_x=0.5
    )
    return sankey

apple_income_statement_metrics_2022()